In [1]:
from policyengine_us.data.datasets.cps.raw_cps import RawCPS_2022, RawCPS_2019

In [2]:
cps_22 = RawCPS_2022()
cps_19 = RawCPS_2019()

Downloaded ASEC: 100%|██████████| 150M/150M [00:45<00:00, 3.27MiB/s]  
